Link del Modelo:
[https://drive.google.com/file/d/1ht8KFE5BXdcv87D90K5eC6yVuOv3IIq6/view?usp=sharing](https://)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install flask-ngrok

In [10]:
import os
import tarfile
tar = tarfile.open('/content/drive/MyDrive/Proyect_Music/midi.tar', 'r:')
tar.extractall()
tar.close()
#Comprobar las carpetas
os.listdir()

def all_notes():
    #Obtiene las notas de los archivos midi
    dicc = []
    for file in glob.glob("/content/midi/*.mid"):
      midi = converter.parse(file)
    for element in midi.recurse():
        if isinstance(element, note.Note):
          dicc.append(str(element.pitch)) #Obtiene el tono de la nota
        elif isinstance(element, chord.Chord):
          dicc.append('.'.join(str(n) for n in element.normalOrder))
    # Obtiene el nombre de los tonos
    pitchnames = sorted(set(item for item in dicc))
    # Crea un diccionario de tonos y numeros enteros
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    return note_to_int
  
def create_midi(prediction_output, idx):
    offset = 0
    output_notes = []
    for item in prediction_output:
        pattern = item[0]
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        offset += 0.5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='gan{}.mid'.format(idx))

def generate(preds):
        idx = 0
        dicc = all_notes()
        pitchnames = dicc.keys()
        print("len pitchnames", len(pitchnames))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        #fake = gen(fixed_noise)
        fake = preds
        fake = fake[0].resize(1, 64*64)
        print("fake", fake[0,100:200])
        pred_notes = fake[0]*34+34
        print("pred desnorm", pred_notes[100:200])
        pred_notes = [int_to_note[int(x)] for x in pred_notes]
        print("pred int", pred_notes)
        create_midi(pred_notes, idx)

In [12]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import torch

app = Flask(__name__)
run_with_ngrok(app)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.jit.load('/content/drive/MyDrive/Proyect_Music/gen.zip')

@app.route("/", methods=['GET','POST'])
def predict():
    preds = model(torch.randn(3, 100, 1, 1).to(device))
    generate(preds)
    return """
        <!doctype html>
          <head><title>Music GAN</title></head>
          <body>
            <div>
              <a href="gan0.mid" download="DCGANmusic.mid">Descargar el audio generado</a>
            </div>
          </body>
        </html>
        """

@app.route('/hola', methods=['GET','POST'])
def hello_world():
    return jsonify('Bienvenido')

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://08c356f42280.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [16/Jun/2021 18:54:08] "GET / HTTP/1.1" 405 -
127.0.0.1 - - [16/Jun/2021 18:54:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [16/Jun/2021 18:54:10] "GET / HTTP/1.1" 405 -


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.jit.load('/content/drive/MyDrive/Proyect_Music/gen.zip')
print(model)
preds = model(torch.randn(3, 100, 1, 1).to(device))
print(preds)

RecursiveScriptModule(
  original_name=Generator
  (dropout): RecursiveScriptModule(original_name=Dropout)
  (net): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(original_name=ConvTranspose2d)
      (1): RecursiveScriptModule(original_name=ReLU)
    )
    (1): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(original_name=ConvTranspose2d)
      (1): RecursiveScriptModule(original_name=ReLU)
    )
    (2): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(original_name=ConvTranspose2d)
      (1): RecursiveScriptModule(original_name=ReLU)
    )
    (3): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(original_name=ConvTranspose2d)
      (1): RecursiveScriptModule(original_name=ReLU)
    )
    (4): RecursiveScriptModule(original_name=ConvTranspose2d)
    (5): RecursiveScriptMo